In [1]:
import cudaq

In [45]:
kernel, int_value, float_value = cudaq.make_kernel(int, float)
print(dir(int_value))

['_QuakeValue__checkTypesAndCreateQuakeValue', '_QuakeValue__floatToVal', '_QuakeValue__intToFloat', '_QuakeValue__intToVal', '__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__rsub__', '__rtruediv__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', 'ctx', 'floatType', 'intType', 'knownUniqueExtractions', 'mlirValue', 'pyKernel', 'size', 'slice']


In [2]:
!nvidia-smi

Sun Jun 23 20:40:16 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:03:00.0 Off |                    0 |
| N/A   30C    P0    63W / 400W |    417MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:41:00.0 Off |                    0 |
| N/A   

cpu 22 qubits

In [23]:
!time python3 ghz.py

{ 0000000000000000000000:516 1111111111111111111111:484 }

real	0m4.962s
user	1m34.670s
sys	0m7.626s


single gpu 10 qubits

In [21]:
!time python3 ghz.py

{ 0000000000:486 1111111111:514 }

real	0m4.306s
user	0m1.581s
sys	0m7.923s


In [28]:
import cudaq


cudaq.set_target("nvidia-mqpu")
qpu_count = cudaq.num_available_gpus()
target = cudaq.get_target()
qpu = target.num_qpus()
print("Number of GPUs:", qpu_count)
print("Number of QPUs:", qpu)

Number of GPUs: 4
Number of QPUs: 4


In [12]:
import cudaq

targets = cudaq.get_targets()
for target in targets:
    print(target)

Target nvidia-mqpu-fp64
	simulator=custatevec_fp64
	platform=mqpu
	description=The NVIDIA MQPU FP64 Target provides a simulated QPU for every available CUDA GPU on the underlying system. Each QPU is simulated via cuStateVec FP64.

Target density-matrix-cpu
	simulator=dm
	platform=default
	description=The Density Matrix CPU Target provides a simulated QPU via OpenMP-enabled, CPU-only density matrix emulation.

Target iqm
	simulator=qpp
	platform=default
	description=

Target tensornet
	simulator=tensornet
	platform=default
	description=

Target nvidia-mqpu
	simulator=custatevec_fp32
	platform=mqpu
	description=The NVIDIA MQPU Target provides a simulated QPU for every available CUDA GPU on the underlying system. Each QPU is simulated via cuStateVec FP32. This target enables asynchronous parallel execution of quantum kernel tasks.

Target qpp-cpu
	simulator=qpp
	platform=default
	description=QPP-based CPU-only backend target

Target remote-mqpu
	simulator=qpp
	platform=mqpu
	description=


In [49]:
qubits = kernel.qalloc(3)
# Prepare qubits in the 1-state.
kernel.x(qubits)

# Get the state of the system. This will execute the provided kernel
# and, depending on the selected target, will return the state as a
# vector or matrix.
kernel(3, 3.14)
cudaq.get_state()
print(state)

AttributeError: 'NoneType' object has no attribute 'name'

In [1]:
import cudaq

cudaq.set_target("nvidia-mqpu")
target = cudaq.get_target()
qpu_count = target.num_qpus()
print("Number of QPUs:", qpu_count)


@cudaq.kernel
def kernel(qubit_count: int):
    qvector = cudaq.qvector(qubit_count)
    # Place qubits in superposition state.
    h(qvector)
    # Measure.
    mz(qvector)


count_futures = []
for qpu in range(qpu_count):
    count_futures.append(cudaq.sample_async(kernel, 5, qpu_id=qpu))

for counts in count_futures:
    print(counts.get())

Number of QPUs: 1
{ 10011:31 01100:32 00011:34 10101:43 00110:34 11111:35 01000:25 11000:25 00100:27 11010:28 00010:29 01101:25 01110:32 01010:32 01111:38 11110:30 10010:37 00111:32 11100:34 10111:35 10000:28 11011:33 10100:31 10110:34 00001:33 10001:21 00000:34 01001:33 11101:27 11001:30 00101:25 01011:33 }

